In [ ]:
import pandas as pd
import numpy as np
import palettable
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
plt.rcParams["font.size"] = "17" # if you don't have arial it will fall back to Deja Vu

In [ ]:
import matplotlib

In [ ]:
from comut import comut

In [ ]:
clinical = pd.read_csv("/add/path/here/EAC_clinical_info.csv",index_col=0)
treatment_mapping = {"Neoadjuvant CROSS": "Neoadj. chemo", "Neoadjuvent carboplatin": "Neoadj. chemo"}
metastatic = (clinical["Tumor?"]=="Yes ") & (clinical["Site"].str.contains("metastasis"))
metastatic.name = "Metastatic?"
clinical["Metastatic?"] = metastatic

clinical["Location"] = clinical["Site"].replace({"GEJ": "Esophagus/GEJ", 
                          "Esophagus": "Esophagus/GEJ"})
clinical["Location"][clinical["Location"].str.contains("Liver")] = "Liver"
clinical["Location"][clinical["Location"].str.contains("Adrenal")] = "Adrenal gland"
clinical["Location"][clinical["Location"].str.contains("Peritoneal")] = "Peritoneum"

clinical["Stage"] = clinical["Grade/stage"].replace({"Stage IV ": "IV", "Stage IV": "IV", 
                                                     "Moderately differentiated; ypT1aN0": "I", 
                                                     "Moderately differentiated; pT1aN0": "I", 
                                                     "Poorly differentiated; ypT2N0": "II", 
                                                     "Presented with stage III became stage IV during esophagectomy when pleural metastases were identified": "III/IV"})

clinical["Treatment"] = ["Neoadj. chemo",
                         "None",
                         "Neoadj. chemo + ICI + RT",
                         "None",
                         "None",
                         "Chemo + HER2 targeted + ICI", 
                         "Neoadj. chemo + HER2 targeted", 
                         "Neoadj. chemo + ICI", 
                         "None",
                         "Neoadj. chemo + VEGFR2i"]

clinical["HER2 status"] = clinical["HER2"].replace({"HER 2 1+": "1+/equivocal"})

clinical = clinical.sort_values(by=["Tumor?","Metastatic?","Location"])

clinical["PD-L1 CPS score"] = [0,2,7,2,3,24,0,8,3,15]

# Comut plot

In [ ]:
pastel_10 = palettable.cartocolors.qualitative.Pastel_10.mpl_colors
prism_10 = palettable.cartocolors.qualitative.Prism_10.mpl_colors
safe_10 = palettable.cartocolors.qualitative.Safe_10.mpl_colors
vivid_10 = palettable.cartocolors.qualitative.Vivid_10.mpl_colors
BluYl_4 = palettable.cartocolors.sequential.BluYl_4.mpl_colors
BrwnYl_4 = palettable.cartocolors.sequential.BrwnYl_4.mpl_colors
Emrld_7 = palettable.cartocolors.sequential.Emrld_7.mpl_colormap

In [ ]:
disease_data = clinical["Tumor?"].replace({"Yes ": "Tumor"}).reset_index()
disease_data["category"] = ["Disease status"]*disease_data.shape[0]
disease_data.columns = ["sample","value","category"]

met_data = clinical["Metastatic?"]
met_data.loc[clinical["Tumor?"].isin(["Adjacent normal"])] = "Not Available"
met_data = met_data.replace({False: "Primary tissue", True: "Metastatic site"}).reset_index()
met_data["category"] = ["Metastatic status"]*met_data.shape[0]
met_data.columns = ["sample","value","category"]
full_dis_data = pd.concat([met_data,disease_data,])
dis_mapping = {"Primary tissue": safe_10[0], 
               "Metastatic site": safe_10[1], 
               "Tumor": safe_10[9], "Adjacent normal": safe_10[3]}

In [ ]:
stage_data = clinical["Stage"].reset_index()
stage_data["category"] = ["Stage"]*stage_data.shape[0]
stage_data.columns = ["sample","value","category"]
stage_mapping = {"I": BluYl_4[0], "II": BluYl_4[1], "III/IV": BluYl_4[2], "IV": BluYl_4[3]}

In [ ]:
site_data = clinical["Location"].reset_index()
site_data["category"] = ["Biopsy site"]*site_data.shape[0]
site_data.columns = ["sample","value","category"]
site_mapping = {"Esophagus/GEJ": vivid_10[0], "Adrenal gland": vivid_10[1], "Liver": vivid_10[4], "Peritoneum": vivid_10[5]}

In [ ]:
her2_data = clinical["HER2 status"].str.rstrip().reset_index()
her2_data["category"] = ["HER2 status"]*her2_data.shape[0]
her2_data.columns = ["sample","value","category"]
her2_mapping = {"0/negative": BrwnYl_4[0], "1+/equivocal": BrwnYl_4[1],
                "2+/equivocal": BrwnYl_4[2], "3+/positive": BrwnYl_4[3]}

In [ ]:
pdl1_data = clinical["PD-L1 CPS score"].reset_index()
pdl1_data["category"] = ["PD-L1 CPS score"]*pdl1_data.shape[0]
pdl1_data.columns = ["sample","value","category"]

In [ ]:
chemo_indicator = ((clinical["Treatment"].str.contains("chemo")) | (clinical["Treatment"].str.contains("Chemo"))).reset_index()
chemo_indicator["category"] = ["Chemotherapy"]*chemo_indicator.shape[0]
chemo_indicator.columns = ["sample","value","category"]
ici_indicator = (clinical["Treatment"].str.contains("ICI")).reset_index()
ici_indicator["category"] = ["ICI"]*ici_indicator.shape[0]
ici_indicator.columns = ["sample","value","category"]
her2_indicator = (clinical["Treatment"].str.contains("HER2")).reset_index()
her2_indicator["category"] = ["HER2 targeted therapy"]*her2_indicator.shape[0]
her2_indicator.columns = ["sample","value","category"]
rt_indicator = (clinical["Treatment"].str.contains("RT")).reset_index()
rt_indicator["category"] = ["Radiotherapy"]*rt_indicator.shape[0]
rt_indicator.columns = ["sample","value","category"]
vegf_indicator = (clinical["Treatment"].str.contains("VEGFR")).reset_index()
vegf_indicator["category"] = ["VEGFR inhibitor"]*vegf_indicator.shape[0]
vegf_indicator.columns = ["sample","value","category"]
treatment_data = pd.concat([her2_indicator,rt_indicator,
                            vegf_indicator,
                            chemo_indicator,ici_indicator])
treatment_mapping = ({True: prism_10[7], False: {'facecolor':'grey', "alpha": 0.2},})

In [ ]:
toy_comut = comut.CoMut()
toy_comut.samples = clinical.index.tolist()

toy_comut.add_categorical_data(data=treatment_data, name="Treatment type", mapping=treatment_mapping)
toy_comut.add_categorical_data(data=naive_indicator, name="Pre-biopsy treatment", mapping=naive_mapping)
toy_comut.add_continuous_data(data=pdl1_data, name="PD-L1 CPS score", mapping=Emrld_7, value_range=(0,25))
toy_comut.add_categorical_data(data=her2_data, name="HER2 status", mapping=her2_mapping)
toy_comut.add_categorical_data(data=site_data, name="Biopsy site", mapping=site_mapping)
toy_comut.add_categorical_data(data=stage_data, name="Stage", mapping=stage_mapping)
toy_comut.add_categorical_data(data=full_dis_data, name="Disease status", mapping=dis_mapping, )
heights = {'Treatment type': 1.7,}

toy_comut.plot_comut(figsize = (8, 8), x_padding = 0.04, 
                     y_padding = 0.04, tri_padding = 0.03, heights = heights)
toy_comut.add_unified_legend(bbox_to_anchor = (1, 1.4), numpoints = 2, ncol = 2)

# color bars must be added manually based on figure coordinates - [left, bottom, width, height]
cps_ax = toy_comut.figure.add_axes([0.97, 0.15, 0.08, 0.014])
# purity ranges from 0 to 1
norm = matplotlib.colors.Normalize(vmin=0, vmax=25)
# create the colorbar with colormap used when the continuous data was added (purp_7)
cps_colorbar = toy_comut.figure.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=Emrld_7),
                                                 cax=cps_ax, orientation='horizontal')
# remove tick marks and move tick labels slightly inwards. Also remove black border
cps_colorbar.ax.tick_params(size=0)
cps_colorbar.set_ticks([0,10,25])
cps_colorbar.set_ticklabels([0,10,25])
cps_colorbar.outline.set_visible(False)

# set title of colorbar to line up with other legend elements
cps_colorbar.set_label('CPS score', labelpad = -50, x = 0.3)
